In [2]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [7]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!',
 'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly controlling.\n\n2. **The word "dough":** This is the main ingredient for pizza crust.  The joke substitutes "dough" for the "do" in "dominated," creating the pun "dough-minated."\n\nTherefore, the pizza maker quit because he was tired of being "dough-minated" - a humorous way of saying he was tired of being controlled, likely by his boss or the demanding nature of the job involving constant work with dough. The humor comes from the unexpected and relevant wordplay.'}

In [9]:

workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly controlling.\n\n2. **The word "dough":** This is the main ingredient for pizza crust.  The joke substitutes "dough" for the "do" in "dominated," creating the pun "dough-minated."\n\nTherefore, the pizza maker quit because he was tired of being "dough-minated" - a humorous way of saying he was tired of being controlled, likely by his boss or the demanding nature of the job involving constant work with dough. The humor comes from the unexpected and relevant wordplay.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-3901-6c58-8002-b93126337346'}}, metadata={'source': 'loop', 'step': 2

In [10]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly controlling.\n\n2. **The word "dough":** This is the main ingredient for pizza crust.  The joke substitutes "dough" for the "do" in "dominated," creating the pun "dough-minated."\n\nTherefore, the pizza maker quit because he was tired of being "dough-minated" - a humorous way of saying he was tired of being controlled, likely by his boss or the demanding nature of the job involving constant work with dough. The humor comes from the unexpected and relevant wordplay.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-3901-6c58-8002-b93126337346'}}, metadata={'source': 'loop', 'step': 

In [11]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti blush? \n\nBecause it saw the meatball, and it was feeling a little saucy!',
 'explanation': 'The joke plays on a few layers of wordplay and double entendre:\n\n* **Blushing:** Blushing is a physical reaction where someone\'s face turns red, usually due to embarrassment, shyness, or feeling attracted to someone.  It\'s a human characteristic.\n\n* **Saucy:**  "Saucy" has two meanings here:\n    * **Literal:** Spaghetti is often served with sauce.\n    * **Figurative:** "Saucy" can also mean cheeky, flirtatious, or slightly inappropriate.\n\n* **The Setup:** The joke sets up a scenario where spaghetti, an inanimate object, is given human-like qualities (the ability to blush).\n\n* **The Punchline:** The punchline reveals the reason for the spaghetti\'s blush: it saw the meatball and was feeling "saucy."  The joke relies on the double meaning of "saucy" to create humor.  The spaghetti is blushing because it\'s feeling a bit flirtatious (

In [13]:
workflow.get_state(config2).config

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f081401-9d9b-6be7-8002-00eac7f607d2'}}

In [19]:
list(workflow.get_state_history(config2))[-1].config

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f081401-82a1-60ab-bfff-38d3dc5be219'}}

## Time Travel

In [22]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly controlling.\n\n2. **The word "dough":** This is the main ingredient for pizza crust.  The joke substitutes "dough" for the "do" in "dominated," creating the pun "dough-minated."\n\nTherefore, the pizza maker quit because he was tired of being "dough-minated" - a humorous way of saying he was tired of being controlled, likely by his boss or the demanding nature of the job involving constant work with dough. The humor comes from the unexpected and relevant wordplay.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-3901-6c58-8002-b93126337346'}}, metadata={'source': 'loop', 'step': 

In [21]:
list(workflow.get_state_history(config1))[1].config

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0813fb-2ad9-6a45-8001-567b5460f172'}}

In [23]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0813fb-2ad9-6a45-8001-567b5460f172"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0813fb-2ad9-6a45-8001-567b5460f172'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-08-24T23:11:42.912979+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-23be-61bf-8000-8bd827779e92'}}, tasks=(PregelTask(id='7d0bc49e-8f19-6eb2-1237-b820d424b155', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly controlling.\n\n2. **The word "dough":** This is the main ingredient for pizza crust.  The

In [24]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0813fb-2ad9-6a45-8001-567b5460f172"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!',
 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-minated!" is a pun. Here\'s why it\'s funny:\n\n* **"Dough-minated" is a play on words.** It combines the word "dough," referring to pizza dough, with the word "dominated," meaning controlled or overpowered.\n\n* **The humor comes from the double meaning.** The joke suggests that the pizza maker was being controlled or bossed around at work ("dominated").  However, the use of "dough" implies that this control was specifically related to his work with pizza dough, making the reason both absurd and relatable to anyone who\'s ever felt overwhelmed at their job.\n\n* **It\'s a simple, clean pun.**  Puns are often humorous because they rely on unexpected wordplay, and this one is easy to understand and appreciate.\n\nEssentially, the joke is funny because it takes a common

In [27]:
workflow.get_state(config1).values

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!',
 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-minated!" is a pun. Here\'s why it\'s funny:\n\n* **"Dough-minated" is a play on words.** It combines the word "dough," referring to pizza dough, with the word "dominated," meaning controlled or overpowered.\n\n* **The humor comes from the double meaning.** The joke suggests that the pizza maker was being controlled or bossed around at work ("dominated").  However, the use of "dough" implies that this control was specifically related to his work with pizza dough, making the reason both absurd and relatable to anyone who\'s ever felt overwhelmed at their job.\n\n* **It\'s a simple, clean pun.**  Puns are often humorous because they rely on unexpected wordplay, and this one is easy to understand and appreciate.\n\nEssentially, the joke is funny because it takes a common

In [31]:
for x in list(workflow.get_state_history(config1))[::-1]:
    print(x.config)
    print(x.values)
    print('-'*20)

{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-23bb-6ad9-bfff-5213aaca9366'}}
{}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-23be-61bf-8000-8bd827779e92'}}
{'topic': 'pizza'}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-2ad9-6a45-8001-567b5460f172'}}
{'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!'}
--------------------
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-3901-6c58-8002-b93126337346'}}
{'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke plays on two things:\n\n1. **The word "dominated":** This word means to be controlled or ruled by someone or something. In a work context, it can imply a boss or work environment that is overly 

### Update State

In [32]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0813fb-23be-61bf-8000-8bd827779e92", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f08141f-b124-6246-8001-79bda0f3946c'}}

In [33]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08141f-b124-6246-8001-79bda0f3946c'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-08-24T23:28:03.362055+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0813fb-23be-61bf-8000-8bd827779e92'}}, tasks=(PregelTask(id='33b8b95a-d150-6f0b-dc2c-3e85f5236bc1', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-minated!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-minated!" is a pun. Here\'s why it\'s funny:\n\n* **"Dough-minated" is a play on words.** It combines the word "dough," referring to pizza

In [34]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f08141f-b124-6246-8001-79bda0f3946c"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it felt like it was being used for its filling!',
 'explanation': 'Okay, here\'s the breakdown of the joke:\n\n*   **The Setup:** "Why did the samosa break up with the chutney?" This sets up a scenario where we\'re anthropomorphizing food, giving them human-like relationships and problems.\n\n*   **The Punchline:** "Because it felt like it was being used for its filling!"\n\n*   **The Humor:** The humor comes from the double meaning of "filling."\n\n    *   **Literal Meaning:** A samosa contains a mixture of ingredients (potatoes, peas, spices, etc.) that is its "filling." This is the obvious connection.\n    *   **Figurative Meaning:** "Feeling used" is an idiom that means someone feels like they are only valued for what they can provide to another person, not for who they are as an individual.\n\nThe joke plays on the fact that the chutney (a condiment often eaten with samosas) is primarily used to

In [35]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it felt like it was being used for its filling!', 'explanation': 'Okay, here\'s the breakdown of the joke:\n\n*   **The Setup:** "Why did the samosa break up with the chutney?" This sets up a scenario where we\'re anthropomorphizing food, giving them human-like relationships and problems.\n\n*   **The Punchline:** "Because it felt like it was being used for its filling!"\n\n*   **The Humor:** The humor comes from the double meaning of "filling."\n\n    *   **Literal Meaning:** A samosa contains a mixture of ingredients (potatoes, peas, spices, etc.) that is its "filling." This is the obvious connection.\n    *   **Figurative Meaning:** "Feeling used" is an idiom that means someone feels like they are only valued for what they can provide to another person, not for who they are as an individual.\n\nThe joke plays on the fact that the chutney (a condiment often eaten with samosas) 

## Fault Tolerance

In [3]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [4]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [5]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
import time
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

In [ ]:
# Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))